In [1]:
!nvidia-smi

Mon Nov 30 17:57:03 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 455.32.00    CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:1D:00.0 Off |                    0 |
| N/A   31C    P0    62W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  Off  | 00000000:1E:00.0 Off |                    0 |
| N/A   

In [2]:
# Standard Libraries
import os
import glob
import shutil
import nvidia_smi
import pickle

# External Dependencies
import cupy as cp
import cudf
import dask_cudf
from dask_cuda import LocalCUDACluster
import dask
from dask.distributed import Client
from dask.utils import parse_bytes
from dask.delayed import delayed
import dask.dataframe as dd
import dask.array as da
import rmm
 
import dask_ml
from dask_ml.preprocessing import OneHotEncoder

from pathlib import Path
import pandas as pd
import numpy as np

# NVTabular
import nvtabular as nvt
import nvtabular.ops as ops
from nvtabular.io import Shuffle
from nvtabular.utils import device_mem_size

import warnings
warnings.filterwarnings('ignore')

import logging

In [ ]:
from fencoding_GPUs import set_cluster_client
client = set_cluster_client(n_gpus=-1, device_spill_frac=0.8)

Dask Version: 2.30.0
Dask cuDF Version: 0.16.0


 n_gpus_avail: 2


Client Scheduler: tcp://127.0.0.1:46829 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 2 Cores: 2 Memory: 404.32 GB



 Dashboard avail: http://localhost:8888/proxy/8787/status


In [4]:
# home-credit-default-risk tables
if not os.path.isfile("data/application_test.csv"):
    %cd data
    ! wget https://www.dropbox.com/s/j9xwcj9ixki5t2l/home-credit-default-risk.zip?dl=0 -O data.zip
    ! unzip -q data.zip
    ! rm data.zip
# default-of-credit-card-clients-dataset
if not os.path.isfile("data/default_ucr.csv"):
    %cd data
    ! wget https://www.dropbox.com/s/lj0d7qez18ea7dx/UCI_Credit_Card.csv?dl=0 -O default_ucr.csv
    %cd ..

In [5]:
# Read in the source datasets
dict_ = {
    'datasets':[
                # default-of-credit-card-clients-datasets
                pd.read_csv('./data/default_ucr.csv'),
    ],

    'name_dropped_columns':
                [
                 # default-of-credit-card-clients-datasets
                 ['ID', 'default.payment.next.month']
                 #['default.payment.next.month'] # 'ID' is needed for shuffling
    ],   
}

In [6]:
N_D = 0
X= dict_['datasets'][N_D]
#X

In [7]:
from fencoding_GPUs import FEncoding_advanced

In [8]:
fencoding = FEncoding_advanced(client)
fencoding.rest_col_names = ['ID']
fencoding.y_names = ['default.payment.next.month']

# Ready, TODO: check for correctness 
fencoding.elim_empty_columns(X)
fencoding.initialize_types(X, return_dtype=False)


 dropped columns: []


{'categor_columns': ['LIMIT_BAL',
  'SEX',
  'EDUCATION',
  'MARRIAGE',
  'AGE',
  'PAY_0',
  'PAY_2',
  'PAY_3',
  'PAY_4',
  'PAY_5',
  'PAY_6',
  'BILL_AMT1',
  'BILL_AMT2',
  'BILL_AMT3',
  'BILL_AMT4',
  'BILL_AMT5',
  'BILL_AMT6',
  'PAY_AMT1',
  'PAY_AMT2',
  'PAY_AMT3',
  'PAY_AMT4',
  'PAY_AMT5',
  'PAY_AMT6'],
 'numer_columns': [],
 'time_columns': []}

In [9]:
fencoding.processing(X, encode_categor_type = 'categorify', fill_with_value = 'extreme_values', 
                     save_to_csv = False)


 extrim_values: {'LIMIT_BAL': 1000000.0, 'EDUCATION': 0, 'MARRIAGE': 0, 'AGE': 79, 'PAY_0': 7, 'PAY_2': 8, 'PAY_3': 8, 'PAY_4': 1, 'PAY_5': 8, 'PAY_6': 8, 'BILL_AMT1': -165580.0, 'BILL_AMT2': -69777.0, 'BILL_AMT3': -157264.0, 'BILL_AMT4': -170000.0, 'BILL_AMT5': -81334.0, 'BILL_AMT6': -339603.0, 'PAY_AMT1': 5361.0, 'PAY_AMT2': 1684259.0, 'PAY_AMT3': 896040.0, 'PAY_AMT4': 621000.0, 'PAY_AMT5': 4683.0, 'PAY_AMT6': 4731.0}
1000000.0 LIMIT_BAL
0 EDUCATION
0 MARRIAGE
79 AGE
7 PAY_0
8 PAY_2
8 PAY_3
1 PAY_4
8 PAY_5
8 PAY_6
-165580.0 BILL_AMT1
-69777.0 BILL_AMT2
-157264.0 BILL_AMT3
-170000.0 BILL_AMT4
-81334.0 BILL_AMT5
-339603.0 BILL_AMT6
5361.0 PAY_AMT1
1684259.0 PAY_AMT2
896040.0 PAY_AMT3
621000.0 PAY_AMT4
4683.0 PAY_AMT5
4731.0 PAY_AMT6


,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,23,2,3,2,21,1,1,1,1,1,...,1091,4734,1964,1808,7289,941,4969,4356,450,0
1,3,2,3,2,14,4,5,5,5,3,...,8083,8096,7959,1,2975,1,947,1796,662,0
2,15,2,3,2,17,3,3,3,3,3,...,9877,4843,5718,3756,3696,3735,2531,2513,3577,0
3,44,2,2,3,17,3,3,3,3,3,...,21348,20636,19935,6476,6519,5922,5034,4571,4435,0
4,21,2,2,3,15,1,2,2,2,2,...,378,461,1387,136,1580,1,139,1267,929,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,22,1,2,3,7,3,3,3,5,4,...,18525,18143,17953,4116,5526,3462,1,3367,3357,0
14996,6,2,3,2,14,2,2,2,2,2,...,990,3597,415,5108,2924,796,4232,1,1,0
14997,11,2,2,2,15,3,3,3,3,3,...,14929,14920,14988,1801,2464,1988,2055,3577,2503,0
14998,4,2,3,3,4,5,5,10,10,9,...,1985,2023,2070,1,1,1,1,1,1,1


In [10]:
X = pd.DataFrame(
    [
        [1,2.2,5.4, 0, 'a',1],
        [10, 5,8, 9.2, 'a', 1],
        [11,4,6,2,'q', 2],
        [-3,4,5,6,'w', 2],
        [None,None,5,6,'w', 2],
        [None,100000,5,6,None, None],
        [1,2.2,5.4, 0, 'a',1],
        [0.2, 5,8, 9.2, 'a', 1],
        [10000,4,6,2,'q', 2],
        [3,4,5,6,'w', 2],
        [None,None,5,6,'w', 2],
        [np.nan,100000,5,6,None, None]
    
    ]
)
X.columns = ['a', '1', '2', '3', '4', '5']
X

,a,1,2,3,4,5
0,1.0,2.2,5.4,0.0,a,1.0
1,10.0,5.0,8.0,9.2,a,1.0
2,11.0,4.0,6.0,2.0,q,2.0
3,-3.0,4.0,5.0,6.0,w,2.0
4,NaN,NaN,5.0,6.0,w,2.0
5,NaN,100000.0,5.0,6.0,None,NaN
6,1.0,2.2,5.4,0.0,a,1.0
7,0.2,5.0,8.0,9.2,a,1.0
8,10000.0,4.0,6.0,2.0,q,2.0
9,3.0,4.0,5.0,6.0,w,2.0


In [11]:
fencoding = FEncoding_advanced(client)
fencoding.initialize_types(X)

{'categor_columns': ['4', '5'],
 'numer_columns': ['a', '1', '2', '3'],
 'time_columns': []}

In [13]:
fencoding.processing(X, 
                     encode_categor_type = 'onehotencoding',
                     outliers_detection_technique = 'iqr_proximity_rule',
                     fill_with_value = 'zeros', 
                     save_to_csv = False)

NameError: name 'OneHotEncoder' is not defined

In [211]:
X=pd.DataFrame([[1,2, np.nan, 1000], 
                [3,4, np.nan, 10], 
                [np.nan, np.nan, 2, 1], 
                [10000, 2, np.nan, 8], 
                [0.1, 3,2], 
                [1,1,2, 11], 
                [1,1,2, 11]])
X.columns = ['0', '1', '2', '3']
X

,0,1,2,3
0,1.0,2.0,NaN,1000.0
1,3.0,4.0,NaN,10.0
2,NaN,NaN,2.0,1.0
3,10000.0,2.0,NaN,8.0
4,0.1,3.0,2.0,NaN
5,1.0,1.0,2.0,11.0
6,1.0,1.0,2.0,11.0


In [212]:
fencoding = FEncoding_advanced(client)
fencoding.initialize_types(X)

{'categor_columns': ['1', '2', '3'],
 'numer_columns': ['0'],
 'time_columns': []}

In [213]:
fencoding.processing(X, 
                     encode_categor_type = None,#'onehotencoding',
                     outliers_detection_technique = 'iqr_proximity_rule',
                     fill_with_value = None,#'zeros', 
                     save_to_csv = False)

column: 0, lower: 1.0, upper: 1.0


,0,1,2,3
0,1.0,3.0,2.0,<NA>
1,1.0,1.0,2.0,11.0
2,1.0,1.0,2.0,11.0
0,1.0,4.0,2.0,1.0
1,1.0,4.0,<NA>,10.0
2,1.0,2.0,2.0,8.0
3,1.0,2.0,<NA>,1000.0


In [206]:
X_cat_encoded

dask.array<_check_and_search_block, shape=(7, 3), dtype=float64, chunksize=(4, 3), chunktype=scipy.csr_matrix>

In [96]:
X = pd.DataFrame([
'Jan 19, 1990',
'January 19, 1990',
'Jan 19,1990',
'01/19/1990',
'01/19/90',
'1990',
'Jan 1990',
'01.02.2000',
'2000.02.01',
'01-02-2000',
'2111-01-01 12:48:20',
'123',
'abs 123', 
1339521878.04,
'1339521878.04'
]).T
X = X.append(X)
X = dd.from_pandas(X, npartitions=2)
X

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
npartitions=1,,,,,,,,,,,,,,,
0,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object
0,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [97]:
import datetime as dt

In [98]:
def pars_date(x):
    fmts = ('%Y', '%b %d, %Y','%b %d, %Y','%B %d, %Y','%B %d %Y','%m/%d/%Y','%m/%d/%y','%b %Y','%B%Y','%b %d,%Y', 
              '%d.%m.%Y', '%Y.%m.%d', '%d-%m-%Y', '%Y-%m-%d %H:%M:%S')
    t = True
    if str(x.dtype) == 'object':
        for fmt in fmts:
            try:
                return dd.Series([dt.datetime.strptime(str(x.iloc[i]), fmt) for i in range(len(x))]).apply(lambda q: q.strftime('%m/%d/%Y')).astype('datetime64[ns]')
                t = False
                break 
            except ValueError:
                pass
    if t and (len(str(x.iloc[0])) > 9) and (len(str(x.iloc[0])) <= 14): 
    # TODO: better condition on string to identify that it is unix timestep
        try:
            x = x.astype('float')
            return pd.Series([dt.datetime.fromtimestamp(x.iloc[i]) for i in range(len(x))]).apply(lambda q: q.strftime('%m/%d/%Y')).astype('datetime64[ns]')
        except ValueError:
            pass

In [99]:
def date_replace_(X):
    for column in X.columns:
        x = pars_date(X[column])
        try: 
            print(x.nunique())
            X[column] = x
        except AttributeError:
            pass
    return X

In [105]:
X[0][0]

NotImplementedError: Series getitem in only supported for other series objects with matching partition structure